In [110]:
pwd

'C:\\Users\\ryanc\\Desktop\\NU (4th) Senior classes\\Stat 390 - Data Science Project\\Code Work'

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import sklearn as sk
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn import impute
from sklearn import preprocessing
from sklearn.impute import KNNImputer
import itertools
import time

In [2]:
# Work from Rakin's Week 4 code
data = pd.read_csv("owid-covid-data.csv")
data.shape

(166326, 67)

In [3]:
# missingness
missing = pd.DataFrame(data.isnull().sum())
missing['% missing'] = data.isnull().sum()/data.shape[0]
missing.columns = ["Missing Count", "Proportion Missing"]
missing = missing.sort_values(by="Proportion Missing", ascending=True)

In [4]:
# Finding number of missing values for each column
data.apply(lambda x: x.isnull().sum())

iso_code                                        0
continent                                    9956
location                                        0
date                                            0
total_cases                                  3033
                                            ...  
human_development_index                     30073
excess_mortality_cumulative_absolute       160630
excess_mortality_cumulative                160630
excess_mortality                           160630
excess_mortality_cumulative_per_million    160630
Length: 67, dtype: int64

In [5]:
missing = missing[missing["Proportion Missing"] < .20]
missing

,Missing Count,Proportion Missing
iso_code,0,0.000000
location,0,0.000000
date,0,0.000000
population,1075,0.006463
total_cases,3033,0.018235
new_cases,3193,0.019197
total_cases_per_million,3791,0.022793
new_cases_per_million,3951,0.023755
new_cases_smoothed,5176,0.031120
new_cases_smoothed_per_million,5928,0.035641


In [6]:
# Work from Sid's Week 5 work (updated on 'Missingness EDA' Jupyter nootbook Python file)
acceptable_columns = list(missing.index.values)
acceptable_columns

['iso_code',
 'location',
 'date',
 'population',
 'total_cases',
 'new_cases',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed',
 'new_cases_smoothed_per_million',
 'continent',
 'life_expectancy',
 'population_density',
 'new_deaths',
 'total_deaths',
 'new_deaths_per_million',
 'total_deaths_per_million',
 'diabetes_prevalence',
 'new_deaths_smoothed',
 'new_deaths_smoothed_per_million',
 'gdp_per_capita',
 'median_age',
 'aged_70_older',
 'cardiovasc_death_rate',
 'aged_65_older',
 'human_development_index']

In [7]:
continent_data = data.groupby("location").count()
continent_data = continent_data.drop(columns=[col for col in continent_data if col not in acceptable_columns])
continent_data

,iso_code,continent,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,life_expectancy,human_development_index
location,,,,,,,,,,,,,,,,,,,,,
Afghanistan,741,741,741,741,739,721,713,713,707,741,...,741,741,741,741,741,741,741,741,741,741
Africa,752,0,752,751,752,746,728,752,746,751,...,752,0,0,0,0,0,0,0,0,0
Albania,740,740,740,727,726,720,725,725,719,727,...,740,740,740,740,740,740,740,740,740,740
Algeria,740,740,740,740,740,734,724,724,718,740,...,740,740,740,740,740,740,740,740,740,740
Andorra,734,734,734,734,734,728,714,714,708,734,...,734,734,0,0,0,0,734,734,734,734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wallis and Futuna,503,503,503,503,503,497,350,350,344,503,...,503,0,0,0,0,0,0,0,503,0
World,774,0,774,774,774,768,774,774,768,774,...,774,774,774,774,774,774,774,774,774,774
Yemen,695,695,695,695,694,682,675,675,669,695,...,695,695,695,695,695,695,695,695,695,695


In [8]:
missing2 = pd.DataFrame(continent_data.sum(axis=1))
missing2['% missing'] = 1 - (missing2/19350)
missing2 = missing2[missing2["% missing"] < .20]
missing2

,0,% missing
location,,
Afghanistan,18301,0.054212
Albania,18304,0.054057
Algeria,18380,0.050129
Angola,17806,0.079793
Antigua and Barbuda,17869,0.076537
...,...,...
Vietnam,18133,0.062894
World,18552,0.041240
Yemen,17215,0.110336


In [9]:
acceptable_locations = list(missing2.index.values)
acceptable_locations

['Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czechia',
 'Democratic Republic of Congo',
 'Denmark',
 'Djibouti',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hong Kong'

In [10]:
new_data = data[data['location'].isin(acceptable_locations)]
new_data = new_data.drop(columns=[col for col in new_data if col not in acceptable_columns])
new_data

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166321,ZWE,Africa,Zimbabwe,2022-03-01,236871.0,491.0,413.000,5395.0,0.0,1.000,...,15092171.0,42.729,19.6,2.822,1.882,1899.775,307.846,1.82,61.49,0.571
166322,ZWE,Africa,Zimbabwe,2022-03-02,237503.0,632.0,416.286,5396.0,1.0,1.143,...,15092171.0,42.729,19.6,2.822,1.882,1899.775,307.846,1.82,61.49,0.571
166323,ZWE,Africa,Zimbabwe,2022-03-03,237503.0,0.0,362.286,5396.0,0.0,0.857,...,15092171.0,42.729,19.6,2.822,1.882,1899.775,307.846,1.82,61.49,0.571
166324,ZWE,Africa,Zimbabwe,2022-03-04,238739.0,1236.0,467.429,5397.0,1.0,0.714,...,15092171.0,42.729,19.6,2.822,1.882,1899.775,307.846,1.82,61.49,0.571


In [11]:
list(new_data.dtypes)

[dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64')]

In [12]:
list(new_data.columns)

['iso_code',
 'continent',
 'location',
 'date',
 'total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_deaths_smoothed_per_million',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'cardiovasc_death_rate',
 'diabetes_prevalence',
 'life_expectancy',
 'human_development_index']

In [13]:
# Dropping the columns with non-numeric values from new_data
new_data_numeric = new_data.drop(columns=['iso_code', 'location', 'date', 'continent'])

# Techniques for imputing on missing data
Website source: https://www.datacamp.com/tutorial/techniques-to-handle-missing-data-values

## Missing At Random: (1) Multiple imputation - MICE (2) Regression imputation
For MAR, the probability of the value being missing is related to the value of the variable or other variables in the dataset. This means that not all the observations and variables have the same chance of being missing. In the context of our dataset and problem, a lot of our missing data comes from low-income or low GDP countries and most notably from the time period of February/March 2020 until April/May 2020.

## Missing Not At Random: (1) Pattern Substitution (2) Maximum Likelihood estimation
MNAR is considered to be the most difficult scenario among the three types of missing data. It is applied when neither MAR nor MCAR apply. In this situation, the probability of being missing is completely different for different values of the same variable, and these reasons can be unknown to us. Here in our case, the most likely reason for our countries having missing data is because testing for COVID had not started yet so the values were truly unknown. Some countries implemented testing sooner than others, and some of them may have withheld their data for a long time before sharing it.

In [14]:
# Filling missing values: Method 1 - Naive way
new_data_naive = new_data_numeric.fillna(method = 'ffill')

In [15]:
# Filling missing values: Method 2 - mputing missing values as the mean of non-missing values
new_data_mean = new_data_numeric.fillna(new_data_numeric.mean())

### KNN
Imputing based on similar values next to or nearby the missing values

In [16]:
# Filling missing values: Method 3 - KNN Imputation

# creating a data frame from the list
Before_imputation = pd.DataFrame(new_data_numeric)

# printing dataset before imputation
print("Data Before performing imputation\n", Before_imputation)

Data Before performing imputation
         total_cases  new_cases  new_cases_smoothed  total_deaths  new_deaths  \
0               5.0        5.0                 NaN           NaN         NaN   
1               5.0        0.0                 NaN           NaN         NaN   
2               5.0        0.0                 NaN           NaN         NaN   
3               5.0        0.0                 NaN           NaN         NaN   
4               5.0        0.0                 NaN           NaN         NaN   
...             ...        ...                 ...           ...         ...   
166321     236871.0      491.0             413.000        5395.0         0.0   
166322     237503.0      632.0             416.286        5396.0         1.0   
166323     237503.0        0.0             362.286        5396.0         0.0   
166324     238739.0     1236.0             467.429        5397.0         1.0   
166325     239019.0      280.0             459.429        5397.0         0.0   

    

In [18]:
# scaling the data
scaler = sk.preprocessing.MinMaxScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(Before_imputation))

In [23]:
# creating an object for KNNImputer
# n neighbors chosen: k=sqrt(n)
imputer = KNNImputer(n_neighbors=408, weights = "uniform")
After_imputation = imputer.fit_transform(scaled_data)

# reverse scaling
unscaled_data = scaler.inverse_transform(After_imputation)

# printing dataset after performing the operation
print("\n\nAfter performing imputation\n",unscaled_data)



After performing imputation
 [[5.00000000e+00 5.00000000e+00 1.41181358e+02 ... 9.59000000e+00
  6.48300000e+01 5.11000000e-01]
 [5.00000000e+00 0.00000000e+00 1.41181358e+02 ... 9.59000000e+00
  6.48300000e+01 5.11000000e-01]
 [5.00000000e+00 0.00000000e+00 1.41181358e+02 ... 9.59000000e+00
  6.48300000e+01 5.11000000e-01]
 ...
 [2.37503000e+05 0.00000000e+00 3.62286000e+02 ... 1.82000000e+00
  6.14900000e+01 5.71000000e-01]
 [2.38739000e+05 1.23600000e+03 4.67429000e+02 ... 1.82000000e+00
  6.14900000e+01 5.71000000e-01]
 [2.39019000e+05 2.80000000e+02 4.59429000e+02 ... 1.82000000e+00
  6.14900000e+01 5.71000000e-01]]


In [24]:
data_imputed = pd.DataFrame(unscaled_data)

In [26]:
data_imputed.columns = Before_imputation.columns
data_imputed

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,life_expectancy,human_development_index
0,5.0,5.0,141.181358,1468.840686,6.490196,6.546564,0.126,0.126,3.544135,36.872676,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
1,5.0,0.0,141.181358,1468.840686,6.490196,6.546564,0.126,0.000,3.544135,36.872676,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
2,5.0,0.0,141.181358,1468.840686,6.490196,6.546564,0.126,0.000,3.544135,36.872676,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
3,5.0,0.0,141.181358,1468.840686,6.490196,6.546564,0.126,0.000,3.544135,36.872676,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
4,5.0,0.0,141.181358,1468.840686,6.490196,6.546564,0.126,0.000,3.544135,36.872676,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130738,236871.0,491.0,413.000000,5395.000000,0.000000,1.000000,15694.959,32.533,27.365000,357.470000,...,15092171.0,42.729,19.6,2.822,1.882,1899.775,307.846,1.82,61.49,0.571
130739,237503.0,632.0,416.286000,5396.000000,1.000000,1.143000,15736.835,41.876,27.583000,357.536000,...,15092171.0,42.729,19.6,2.822,1.882,1899.775,307.846,1.82,61.49,0.571
130740,237503.0,0.0,362.286000,5396.000000,0.000000,0.857000,15736.835,0.000,24.005000,357.536000,...,15092171.0,42.729,19.6,2.822,1.882,1899.775,307.846,1.82,61.49,0.571
130741,238739.0,1236.0,467.429000,5397.000000,1.000000,0.714000,15818.731,81.897,30.972000,357.603000,...,15092171.0,42.729,19.6,2.822,1.882,1899.775,307.846,1.82,61.49,0.571


In [32]:
# checking for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

X = add_constant(data_imputed)

vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

for i in range(len(X.columns)):
    vif_data.loc[i,'VIF'] = variance_inflation_factor(X.values, i)

print(vif_data)

                            feature         VIF
0                             const  326.722820
1                       total_cases   59.880148
2                         new_cases   36.513919
3                new_cases_smoothed   42.288050
4                      total_deaths   61.535532
5                        new_deaths   31.100433
6               new_deaths_smoothed   38.907473
7           total_cases_per_million    3.952718
8             new_cases_per_million    4.277321
9    new_cases_smoothed_per_million    5.439118
10         total_deaths_per_million    3.177577
11           new_deaths_per_million    3.015312
12  new_deaths_smoothed_per_million    3.531753
13                       population    4.711494
14               population_density    1.240140
15                       median_age   17.402926
16                    aged_65_older  115.283581
17                    aged_70_older  104.841754
18                   gdp_per_capita    3.095084
19            cardiovasc_death_rate    1